We run this notebook on a GCP cluster with 2 worker nodes. We want to test for the computation time for the LDA algorithm with 2 worker nodes. 

## Importing Necessary Libraries ##

In [1]:
import nltk # test
nltk.download('all') # uncomment this for the first to check that all data has already been nstalled
sc.defaultParallelism

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package cess_cat is already up-

[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package sinica_treebank is already up-to-date!
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /usr/local/share/nltk_data...
[nltk_data]    |   Package smultron is already up-to-date!
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /usr/local/share/nltk_data.

4

In [2]:
import numpy as np

In [3]:
!pip install snscrape

## Getting the Tweets using snscrape ##

In [ ]:
import snscrape.base
import snscrape.modules.twitter as sntwitter
import pandas as pd
# Setting variables to be used below
maxTweets = 100000
# Creating list to append tweet data to
tweets_list2 = []
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('coronaviruspandemic lang:en').get_items()):
    if i>maxTweets:
        break
    tweets_list2.append([tweet.content])
# Creating a dataframe from the tweets list above
tweets_df2 = spark.createDataFrame(tweets_list2, ['words'])
# Display first 5 entries from dataframe
tweets_df2.head()

Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=coronaviruspandemic+lang%3Aen&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaIgKmt_L2IuSYWisCqnbmb7c0mEnEV5O1hFYCJehgHREVGQVVMVDUBFYwDFQAA&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: ChunkedEncodingError(ProtocolError("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer')),), retrying
Er

Row(words='Stay Safe Stay Healthy\n#cgfightscorona \n\n#corona #coronaindia #coronatime #coronawarriors #coronaviruspandemic #covid_19 #covid #covidー19 #india #chhattisgarh #ChhattisgarhNews https://t.co/O3m9KdAfbx')

## Parsing the Data ##

**Converting the tweets into an RDD**

In [5]:
rdd=tweets_df2.rdd
rdd.take(1)

[Row(words='Stay Safe Stay Healthy\n#cgfightscorona \n\n#corona #coronaindia #coronatime #coronawarriors #coronaviruspandemic #covid_19 #covid #covidー19 #india #chhattisgarh #ChhattisgarhNews https://t.co/O3m9KdAfbx')]

**Cleaning and lemmatizing the words in each tweet using the NLTK library**

In [6]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
custom_stop_words = set(['coronaviruspandemic','https','coronavirus', 'corona'])
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer() # see https://www.nltk.org/_modules/nltk/stem/wordnet.html for details

def get_tokens(line):
    ###
    import nltk
#     nltk.download('all')
    ###
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    words = [w for w in words if not w in stop_words]
    # filter out custom stop words
    words = [w for w in words if not w in custom_stop_words]

    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    words = [lmtzr.lemmatize(w) for w in words]
    return (words)

rdd1= rdd.map(lambda r: (r[0], get_tokens(r[0])))
rdd1.take(1)

[('Stay Safe Stay Healthy\n#cgfightscorona \n\n#corona #coronaindia #coronatime #coronawarriors #coronaviruspandemic #covid_19 #covid #covidー19 #india #chhattisgarh #ChhattisgarhNews https://t.co/O3m9KdAfbx',
  ['stay',
   'safe',
   'stay',
   'healthy',
   'cgfightscorona',
   'coronaindia',
   'coronatime',
   'coronawarriors',
   'covid',
   'india',
   'chhattisgarh',
   'chhattisgarhnews'])]

Now, we will remove all the words that appear more than 1000 times in the data

In [7]:
doc_stop_words = rdd1.flatMap(lambda r: r[1]).map(lambda r: (r,1)).reduceByKey(lambda a,b: a+b)

# doc_stop_words.collect()

doc_stop_words = doc_stop_words.filter(lambda a: a[1]>1000).map(lambda r: r[0]).collect()

# throw away stop words and words that are just single letters.
rdd1 = rdd1.map(lambda r: (r[0],[w for w in r[1] if not w in doc_stop_words and not len(w)==1])) 
#                    .filter(lambda row: len(row[1]) > 0)

rdd1.take(1)[0][1][:10]

['healthy',
 'cgfightscorona',
 'coronaindia',
 'coronatime',
 'coronawarriors',
 'chhattisgarh',
 'chhattisgarhnews']

## Converting tokens into Sparse Vectors ##

**Converting the RDD with cleaned and tokenized text into a Spark DataFrame**

In [8]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField, StringType, ArrayType


df = spark.createDataFrame(rdd1,['text','words'])
df.cache()
df.take(1)

[Row(text='Stay Safe Stay Healthy\n#cgfightscorona \n\n#corona #coronaindia #coronatime #coronawarriors #coronaviruspandemic #covid_19 #covid #covidー19 #india #chhattisgarh #ChhattisgarhNews https://t.co/O3m9KdAfbx', words=['healthy', 'cgfightscorona', 'coronaindia', 'coronatime', 'coronawarriors', 'chhattisgarh', 'chhattisgarhnews'])]

In [9]:
cv = CountVectorizer(inputCol="words", outputCol="features", minDF=2)

cv_model = cv.fit(df)

df_w_features = cv_model.transform(df)
df_w_features.cache()
df_w_features.show(10)

+--------------------+--------------------+--------------------+
|                text|               words|            features|
+--------------------+--------------------+--------------------+
|Stay Safe Stay He...|[healthy, cgfight...|(27963,[250,498,1...|
|LIVE: After Kumbh...|[kumbh, uttarakha...|(27963,[332,1422,...|
|Hope viruse Will ...|[hope, viruse, re...|(27963,[38,154,13...|
|ReliefLine is ava...|[reliefline, avai...|(27963,[58,69,114...|
|PM #NarendraModi ...|[lauds, effort, u...|(27963,[52,357,15...|
|@angadku11555552 ...|[pappuyadavjapl, ...|(27963,[9,115,188...|
|The #EU and #Indi...|[eu, set, bolster...|(27963,[20,111,16...|
|@vijayshankar003 ...|[pappuyadavjapl, ...|(27963,[8,59,376,...|
|After the Pandemi...|[fate, every, fam...|(27963,[1,3,7,130...|
|#SurajyaAbhiyan

...|[surajyaabhiyan, ...|(27963,[31,245,26...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



## Latent Dirichlet Allocation ##

We are identifying 5 topics using LDA 

In [11]:
from pyspark.ml.clustering import LDA
import time

t0= time.time()
lda = LDA(k=5, maxIter=10)

lda_model = lda.fit(df_w_features)
print('Time taken:', time.time()-t0)

Time taken: 10.60512900352478
